In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Check if the JDBC driver exists
jdbc_driver_path = "/home/user/work/jars/postgresql-42.7.3.jar"
if not os.path.isfile(jdbc_driver_path):
    raise FileNotFoundError(f"The JDBC driver was not found at the specified path: {jdbc_driver_path}")

# Initialize the Spark session
spark = SparkSession.builder \
    .appName("PostgreSQL to PySpark") \
    .config("spark.jars", jdbc_driver_path) \
    .getOrCreate()

# PostgreSQL connection properties
jdbc_url = "jdbc:postgresql://postgres-staging:5432/staging_db"
connection_properties = {
    "user": "admin",
    "password": "admin",
    "driver": "org.postgresql.Driver"
}

24/06/22 18:34:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/22 18:34:24 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/22 18:34:24 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


## Top Employee by Revenue

In [3]:
# Load necessary tables (assuming they are already loaded as shown above)
order_details = spark.read.jdbc(url=jdbc_url, table="order_details", properties=connection_properties)
orders = spark.read.jdbc(url=jdbc_url, table="orders", properties=connection_properties)
employees = spark.read.jdbc(url=jdbc_url, table="employees", properties=connection_properties)

# Perform the equivalent operations in PySpark
employee_revenue = orders.join(order_details, orders["orderID"] == order_details["orderID"], "inner") \
    .groupBy(
        F.date_format(F.date_trunc("month", orders["orderDate"]), "yyyy-MM-dd").alias("month"),
        orders["employeeID"]
    ).agg(
        F.sum((order_details["unitPrice"] - (order_details["unitPrice"] * order_details["discount"])) * order_details["quantity"]).alias("total_gross_revenue")
    )

# Rank employees based on total gross revenue
ranked_employees = employee_revenue.withColumn(
    "employee_rank",
    F.rank().over(Window.partitionBy("month").orderBy(F.desc("total_gross_revenue")))
).filter(
    F.col("employee_rank") == 1
)

# Join with employees to get final result
final_result = ranked_employees.join(
    employees, ranked_employees["employeeID"] == employees["employeeID"], "inner"
).select(
    ranked_employees["month"],
    ranked_employees["employeeID"],
    F.concat(employees["lastName"], F.lit(", "), employees["firstName"]).alias("employeeName"),
    ranked_employees["total_gross_revenue"]
)

# Show or save the final result as needed
final_result.orderBy(F.asc("month"), F.desc("total_gross_revenue")).show()


+----------+----------+-----------------+-------------------+
|     month|employeeID|     employeeName|total_gross_revenue|
+----------+----------+-----------------+-------------------+
|1996-07-01|         4|Peacock, Margaret|       11860.450000|
|1996-08-01|         8|  Callahan, Laura|        8263.800000|
|1996-09-01|         1|   Davolio, Nancy|        6651.000000|
|1996-10-01|         4|Peacock, Margaret|       13718.970000|
|1996-11-01|         4|Peacock, Margaret|       11311.420000|
|1996-12-01|         5| Buchanan, Steven|       10030.820000|
|1997-01-01|         4|Peacock, Margaret|       23736.465000|
|1997-02-01|         4|Peacock, Margaret|       12121.995000|
|1997-03-01|         3| Leverling, Janet|       11599.400000|
|1997-04-01|         4|Peacock, Margaret|       13475.990000|
|1997-05-01|         3| Leverling, Janet|       18049.600000|
|1997-06-01|         2|   Fuller, Andrew|        6882.200000|
|1997-07-01|         1|   Davolio, Nancy|       19530.930000|
|1997-08